# PHYS6318: Lecture 7

In [9]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
from matplotlib import rcParams
rcParams['animation.html'] = 'jshtml'
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import netCDF4
import time
import os
import glob

## Visualization routines from previous lecture

#### Transport

In [21]:
def calc_transport(u, v, h):
    
    uh = np.zeros((53, 53))
    vh = np.zeros((53, 53))
    
    for j in range(1, 52): 
        for k in range(1, 52):
            uh[k, j] = u[k, j]*0.5*(h[k,j] + h[k+1, j])
            vh[k, j] = v[k, j]*0.5*(h[k,j] + h[k, j+1])

    return uh, vh

In [33]:
def animate_transport(filename):
    ds = xr.open_dataset(filename)
    fig, ax = plt.subplots(figsize=(8,8))

    # show time in units of days
    t = (ds.time - ds.time[0]).values / np.timedelta64(1, 'D')

    ax.contour(ds.xh/1e3, ds.yh/1e3, ds.h[0].T, colors='k', linewidths=0.5)
    
    # compute grid centres
    xu, yu = np.meshgrid(ds.xu, ds.yu)
    xv, yv = np.meshgrid(ds.xv, ds.yv)
    xc = 0.5*(xu + xv)
    yc = 0.5*(yu + yv)
    
    uh, vh = calc_transport(ds.u[0].values, ds.v[0].values, ds.h[0].values)
    
    q = ax.quiver(xc[::3, ::3]/1e3,
                  yc[::3, ::3]/1e3, 
                  uh[::3, ::3].T, 
                  vh[::3 ,::3].T, 
                  scale=70)
    text = ax.text(4, 5.3, f'time= {t[0]:.0f} days')

    def init():
        ax.set_title('Transport')
        ax.set_xlabel('x (km)')
        ax.set_ylabel('y (km)')
        
        return q, text
    
    def update(frame):

        uh, vh = calc_transport(ds.u[frame].values, ds.v[frame].values, ds.h[frame].values)
        
        q.set_UVC(uh[::3, ::3].T, vh[::3, ::3].T)
        text.set_text(f'time= {t[frame]:.1f} days')
        
        return q, text
    
        
    anim = FuncAnimation(fig, update, blit=True,
                         init_func=init,
                         frames=np.arange(0, len(ds.time), 5),
                        )
    plt.close(fig)
    
    return anim


## OMB 5.9 Exercise 13: Inclusion of Nonlinear Terms


### 5.9.1 Aim

The aim of this exercise is to include the nonlinear terms (advection of momentum)
in the shallow-water equations.

### 5.9.2 Formulation of the Nonlinear Terms

Using the product rule of differentiation, the (horizontal) nonlinear terms in our
shallow-water model can be written as:

$$ \mbox{Adv}_h(\xi) = u \frac{\partial \xi}{\partial x} + v \frac{\partial \xi}{\partial y} = \frac{\partial (u \xi)}{\partial x} + \frac{\partial(v  \xi)}{\partial y} - \xi \left( \frac{\partial u}{\partial x} + \frac{\partial u}{\partial x}  \right)$$

where $\xi$ is either $u$ or $v$. The first two terms on the right-hand side of this equation
can be discretised using the TVD advection schemes for a control volume as in
Exercise 11. The remaining term can be formulated in an explicit manner.

### 5.9.3 Sample Code

The code has been modified to output a NetCDF as output.

### 5.9.4 Results

> Inclusion of the nonlinear terms modifies the lake’s circulation (Fig. 5.15). A clockwise eddy establishes in the northwestern corner of the lake and the eddy northwest
of the island has largely disappeared. The upstream scheme is highly diffusive and
therefore triggers rapid establishment of a steady-state circulation in the lake. In
contrast to this, less diffusive TVD advection schemes based on either the Superbee
or the Super-C limiters lead to slight oscillations of the circulation pattern presumably triggered by the initial adjustment of the wind field. Due to reduced numerical
diffusion, either of these schemes should be applied for the nonlinear terms. Use
of the Lax-Wendroff scheme for the nonlinear terms did not lead to satisfactory
results.

In [42]:
filename = 'src/OMB_Ex13/output.nc'

In [43]:
animate_transport(filename)

In class activitiy: explore the different TVD schemes on this problem.